In [ ]:
# Imports
import requests
import json
import csv
import psycopg2
import time
from dotenv import dotenv_values
from openai import AzureOpenAI

In [ ]:
config = dotenv_values("../.env")

In [ ]:
# Database connection settings
DB_HOST = config['DB_HOST']
DB_PORT = config['DB_PORT']
DB_NAME = config['DB_NAME']
DB_USER = config['DB_USER']
DB_PASS = config['DB_PASS']

In [ ]:
# Example row data
# conversation_data = {
#     "conversationID": "36eba05a-aad9-41d4-8bff-037b2ea1bf78",
#     "transcript": "Customer: Hi, I need help with my account. Bot: Sure, I can help!",
#     "customer_sentiment": "positive",
#     "dominant_customer_emotion": "happy",
#     "customer_sentiment_confidence": 0.9231,
#     "date": "30/01/2025",
#     "bot_summary": "Bot handled login issues.",
#     "agent_summary": None,
#     "notes": "Customer seemed satisfied.",
#     "topics": ["account", "login"],
#     "keywords": ["help", "account", "login"]
# }

# Insert query
insert_query = """
    INSERT INTO conversations (
        conversationID,
        userID,
        transcript,
        customer_sentiment,
        dominant_customer_emotion,
        customer_sentiment_confidence,
        date,
        notes,
        topics,
        keywords
    )
    VALUES (
        %(conversationID)s,
        %(userID)s,
        %(transcript)s,
        %(customer_sentiment)s,
        %(dominant_customer_emotion)s,
        %(customer_sentiment_confidence)s,
        %(date)s,
        %(notes)s,
        %(topics)s,
        %(keywords)s
    )
"""

In [ ]:
# Load transcripts
def load_transactions_csv():
    with open("transcripts.csv", mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='\t')
        contents = []
        for row in reader:
            contents.append(row)
    return contents

transcripts = load_transactions_csv()

In [ ]:
# Load system prompt form instructions.md file
with open("instructions.md", mode='r', newline='', encoding='utf-8') as file:
    system_prompt = file.read().strip()

In [ ]:
# Create OpenAI Client
client = AzureOpenAI(
    # https://learn.microsoft.com/azure/ai-services/openai/reference#rest-api-versioning
    api_version=config["AZURE_OPENAI_VERSION"],
    api_key=config["AZURE_OPENAI_API_KEY"],
    # https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=config["AZURE_OPENAI_BASE_URL"],
)

# Function to call Azure GenAI Model and get analysis on the transcript
def get_analytics(transcript):
    try:
        completion = client.chat.completions.create(
            model=config["AZURE_OPENAI_MODEL_NAME"],
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": transcript,
                },
            ],
        )
        data = completion.to_json()
        analysis_str = json.loads(data)['choices'][0]['message']['content']
        return json.loads(analysis_str)
    
    except requests.exceptions.RequestException as e:
        print("Error:", e)

In [ ]:
def write_to_table(data):
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )
    cur = conn.cursor()
    
    # Execute insert
    cur.execute(insert_query, data)
    conn.commit()
    
    print("Row inserted successfully!")
    
    # Clean up
    cur.close()
    conn.close()

In [ ]:
# For each transcript, call GenAI and write analysis results to database
for idx, trans in enumerate(transcripts):
    completion = get_analytics(transcripts[idx]['transcript'])

    print(completion['overall']['dominant_customer_emotion'])

    conversation_data = {
        "conversationID": transcripts[idx]['conversationId'],
        "userID": transcripts[idx]['userId'],
        "transcript": transcripts[idx]['transcript'],
        "customer_sentiment": completion['overall']['customer_sentiment'],
        "dominant_customer_emotion": completion['overall']['dominant_customer_emotion'],
        "customer_sentiment_confidence": completion['overall']['customer_sentiment_confidence'],
        "date": transcripts[idx]['date'],
        "notes": completion['notes'],
        "topics": [c["label"] for c in completion['topics']],
        "keywords": [keyword for topic in completion['topics']for keyword in topic['keywords_detected'] ]
    }
    write_to_table(conversation_data)
    time.sleep(10)